In [1]:
import time; print(time.strftime("The last update of this file: %Y-%m-%d %H:%M:%S", time.gmtime()))
import sys, time
# Establish connection with the UE4 game
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

# Checking status of connection between UnrealCV and UE4 game

res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print('status: ',res)

INFO:__init__:211:Got connection confirm: b'connected to test_game_4_19'


The last update of this file: 2019-06-26 10:40:13
status:  Is Listening
Client Connected
9000
Configuration
Config file: C:/Program Files/Epic Games/UE_4.19/Engine/Binaries/Win64/unrealcv.ini
Port: 9000
Width: 640
Height: 480
FOV: 90.000000
EnableInput: true
EnableRightEye: false



In [72]:
import numpy as np

get_camera_location = client.request('vget /camera/0/location')
get_camera_rotation = client.request('vget /camera/0/rotation')

get_camera_location = get_camera_location.split(" ")
get_camera_location_array = np.array(get_camera_location)
get_camera_location_array = get_camera_location_array.astype(np.float)

get_camera_rotation = get_camera_rotation.split(" ")
get_camera_rotation_array = np.array(get_camera_rotation)
get_camera_rotation_array = get_camera_rotation_array.astype(np.float)

In [73]:
print(get_camera_location_array)
print(get_camera_rotation_array)

[223.851  -0.344  79.887]
[  -0.331 -170.286   -0.   ]


In [74]:
# set_camera_location = client.request('vset /camera/0/location '+str(loc_x)+' '+str(loc_y)+' '+str(loc_z)+'')
# set_camera_rotation = client.request('vset /camera/0/rotation '+str(rot_x)+' '+str(rot_y)+' '+str(rot_z)+'')

set_camera_location = client.request('vset /camera/0/location '+str(get_camera_location_array[0])+' '+str(get_camera_location_array[1])+' '+str(get_camera_location_array[2])+'')
set_camera_rotation = client.request('vset /camera/0/rotation '+str(get_camera_rotation_array[0])+' '+str(get_camera_rotation_array[1])+' '+str(get_camera_rotation_array[2])+'')

In [75]:
print(set_camera_location)
print(set_camera_rotation)

ok
ok


In [76]:
actor_array = ['SM_DenkMitEdelstahlReinigerSpray_18','SM_CalgonitFinishKlarspueler_3','SM_CalgonitFinishVorratspack_12']
actor_array = np.array(actor_array)

In [77]:
for i in actor_array:
    print(i)

SM_DenkMitEdelstahlReinigerSpray_18
SM_CalgonitFinishKlarspueler_3
SM_CalgonitFinishVorratspack_12


In [78]:
dirName = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/new_image/'
viewmode_1 = 'lit'
viewmode_2 = 'object_mask'
viewmode_3 = 'depth'
image_type = '.png'
depth_type = '.npy'

In [79]:
res_lit = client.request('vget /camera/0/'+str(viewmode_1)+str(" ")+str(dirName)+'lit_new'+str(image_type))
res_mask = client.request('vget /camera/0/'+str(viewmode_2)+str(" ")+str(dirName)+'mask_new'+str(image_type))

In [226]:
print(res_lit)
print(res_mask)

F:/unreal_cv_documentation/detect_ROI_for_multi_actor/new_image/lit_new.png
F:/unreal_cv_documentation/detect_ROI_for_multi_actor/new_image/mask_new.png


In [215]:
f = open(str(dirName)+'color_info.txt', 'a')
f = open(str(dirName)+'color_info.txt', 'r+')
f.truncate(0)

for j in actor_array:
    print(get_mask_color)
    get_mask_color= client.request('vget /object/'+str(j)+'/color')
    ww = str(get_mask_color)
    ww=ww.replace('R','')
    ww=ww.replace('G','')
    ww=ww.replace('B','')
    ww=ww.replace('A','')
    ww=ww.replace('=','')
    ww=ww.replace('(','')
    ww=ww.replace(')','')
    ww=ww.replace(',',' ')
    f.write(ww)
    f.write("\n")
    
f.close()

(R=255,G=127,B=0,A=255)
(R=0,G=127,B=127,A=255)
(R=0,G=127,B=255,A=255)


In [216]:
import numpy as np
store=[]
with open(str(dirName)+'color_info.txt') as f1:
    a=f1.readlines()
    for i in a:
        k=i.split('\n')
        m=k[0].split(' ')
        store.append(m)
store=np.array(store,dtype=int)

In [217]:
print(type(store))

<class 'numpy.ndarray'>


In [218]:
store

array([[  0, 127, 127, 255],
       [  0, 127, 255, 255],
       [255, 127,   0, 255]])

In [219]:
r,g,b,a=[store[:,i] for i in range(len(store[0]))]

In [225]:
for in_actor,i in enumerate(actor_array):
    print(r[in_actor])
    print(g[in_actor])
    print(b[in_actor])
    print('\n')

0
127
127


0
127
255


255
127
0




In [248]:
import cv2
import copy

lit_im = cv2.imread(res_lit)
mask_im = cv2.imread(res_mask)
img = np.zeros((mask_im.shape[0],mask_im.shape[1],mask_im.shape[2]), np.uint8)
test_copy_img = copy.copy(img)
test_copy_img_color = copy.copy(img)
test_copy_image_color_mask = copy.copy(test_copy_img_color)

for in_actor,i in enumerate(actor_array):
    for rows in range (0, mask_im.shape[0]):
        for cols in range(0, mask_im.shape[1]):
            if mask_im[rows][cols][0]==b[in_actor] and mask_im[rows][cols][1]==g[in_actor] and mask_im[rows][cols][2]==r[in_actor]:
                test_copy_img[rows][cols] = 255
                test_copy_img_color[rows][cols]=mask_im[rows][cols]

test_copy_image_color_mask = copy.copy(test_copy_img_color)                
                
imgray=cv2.cvtColor(test_copy_img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
image, contours, hierarchy =  cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

roi_list = []

for t in range (0,len(actor_array),1):
    if len(contours)>0:
        print('length is: ',len(contours))
        cnt = contours[t]
        x,y,w,h = cv2.boundingRect(cnt)
        roi_rgb_image=cv2.rectangle(lit_im,(x,y),(x+w,y+h),(0,255,0),1)
        print('info of ',t,' contour ',x,',',y,',',x+w,',',y+h)
        x=x
        y=y
        x_1=x+w
        y_1=y+h
        
        roi_point = [x,y,x_1,y_1]
        roi_list.append(roi_point)

for i in roi_list:
    x_new = i[0]
    y_new = i[1]
    x_w_new = i[2]
    y_h_new = i[3]
    
    roi_black_mask = cv2.rectangle(test_copy_image_color_mask,(x_new,y_new),(x_w_new,y_h_new),(0,255,0),1)

    

f = open(str(dirName)+'roi_info.txt', 'a')
f = open(str(dirName)+'roi_info.txt', 'r+')
f.truncate(0)

for i in roi_list:
    ww = str(i)
    ww=ww.replace('[','')
    ww=ww.replace(']','')
    ww=ww.replace(',','')
    f.write(ww)
    f.write("\n")
f.close()

cv2.imshow('black',img)
# cv2.imshow('lit',lit_im)
# cv2.imshow('mask',mask_im)
cv2.imshow('new_image_black',test_copy_img)
# cv2.imshow('test_copy_img_color', test_copy_img_color)
# cv2.imshow('imgray', imgray)
# cv2.imshow('image_after_thresholding', image)
# cv2.imshow('roi_rgb', roi_rgb_image)
# cv2.imshow('roi_black_mask', roi_black_mask)

if cv2.waitKey() == ord('q'): #press q to close the output image window
        cv2.destroyAllWindows()

length is:  3
info of  0  contour  391 , 288 , 431 , 365
length is:  3
info of  1  contour  281 , 282 , 310 , 373
length is:  3
info of  2  contour  487 , 278 , 546 , 352


In [261]:
test_copy_black_mask_1 = copy.copy(img)
test_copy_black_mask_2 = copy.copy(test_copy_img)
mask = cv2.bitwise_or(test_copy_black_mask_1, test_copy_black_mask_2)

lit_copy = copy.copy(lit_im)

black_rgb = cv2.bitwise_and(lit_copy, lit_copy, mask = mask)

cv2.imshow('black_rgb', black_rgb)

cv2.imshow('mask', mask)

if cv2.waitKey() == ord('q'): #press q to close the output image window
        cv2.destroyAllWindows()

error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\core\src\arithm.cpp:241: error: (-215) (mtype == 0 || mtype == 1) && _mask.sameSize(*psrc1) in function cv::binary_op


In [ ]:
mask.types()

In [256]:
type(lit_im)

numpy.ndarray